In [1]:
import ISO_Networksii as ISON
import ISO_Networks as ISON1
import datawrangler as dtwrnglr
import re
import operator
import string
from collections import Counter
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pandas as pd

# Load Raw Data

In [2]:
U, F, T, = ISON.load_universe()

In [3]:
F1, experiment= ISON.genre_sorted_data(F)

[('Action', 290), ('Comedy', 265), ('Drama', 254), ('Crime', 96), ('Adventure', 61), ('Horror', 56), ('Mystery', 12), ('Animation', 10), ('Fantasy', 6), ('Thriller', 6), ('Sci-Fi', 5), ('Biography', 3), ('Romance', 2), ('Western', 2), ('Family', 1), ('Short', 1)]
['Action', 'Comedy', 'Drama', 'Crime', 'Adventure', 'Horror']


/Users/ViVeri/Desktop/ISO_Networks/ISO_Networksii.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  new['documents'] = [film_strip(i) for i in new.text.tolist()]


In [4]:
experiment

['Action', 'Comedy', 'Drama', 'Crime', 'Adventure', 'Horror']

In [5]:
def genre_binary(Universe, experiment):
    genre_binary_labels= {}
    for key in experiment:
        if key not in genre_binary_labels.keys():
            _, labels =ISON.genre_binary_sorted_data(Universe, key)
            genre_binary_labels[key] = labels 
    return genre_binary_labels
        

In [6]:
data=F1.documents.tolist()
labels = F1['labels'].tolist()
print('The initial genre distribution is:',Counter(labels))
labels_binary=genre_binary(F1,experiment)

The initial genre distribution is: Counter({'Action': 290, 'Comedy': 265, 'Drama': 254, 'Crime': 96, 'Adventure': 61, 'Horror': 56, 'Other': 48})


In [7]:
%time cv, data_N=ISON.BoW_ngram(data,1)

CPU times: user 10 s, sys: 315 ms, total: 10.4 s
Wall time: 11.2 s


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RandomizedLogisticRegression
from sklearn import preprocessing
from sklearn import svm, linear_model
from sklearn.linear_model import SGDClassifier

In [9]:
from sklearn.model_selection import cross_val_score
def train_test(data, labels, test_size):
    X_train, X_test, y_train, y_test = train_test_split(data, labels,
                                                    stratify=labels, 
                                                    test_size=test_size)
    return X_train, X_test, y_train, y_test

def ISON_ngram_Film_experiment(universe_representation, labels, test_size):
    
    
    #STEP 1: SPLIT THE DATA
    X_train, X_test, y_train, y_test=train_test(universe_representation,labels,test_size)
    print('START')
    
    #STEP 2 : INTIALIZE LOGISTIC REGRESSSION
    model_ML = Pipeline([
    ('clf',LogisticRegression(solver='sag')) 
                                                ])
    print('Fitting corpus into LogReg() model')
    print('Corpus is', str(universe_representation.shape))
    
    #STEP 3: TRAIN MODEL
    clf=model_ML.fit(X_train, y_train)
    
    #STEP 4: PREDICT TEST
    y_pred = clf.predict(X_test)
    print('\n')
    
    #STEP 5: CLASSIFACTION REPORT 
    report =classification_report( y_test, y_pred)
    print(report)
    
    print((y_pred == y_test).mean())
#     print(report_classfication(report)   ) 

    #STEP 6: CROSS VALIDATION SCORE 
    scores=cross_val_score(model_ML, X_train, y_train, cv=10)
    print(scores)
    print(np.mean(scores))
    
    return model_ML, clf



def ISON_Film_experiment2(universe_representation, labels, test_size, vectorizer):
    ##This one runs Randomized LogRegression unlike the first, takes a bit longer 
    
    #STEP 1: SPLIT THE DATA
    X_train, X_test, y_train, y_test=train_test(universe_representation,labels,test_size)
    print('START')
    
    #STEP 2: 
    RLR = RandomizedLogisticRegression()
    le = preprocessing.LabelEncoder()
    label_encoded=le.fit_transform(labels)
    randomizedLRM=RLR.fit(universe_representation, label_encoded)
    
    
    feature_scores=randomizedLRM.all_scores_
    index = np.where(feature_scores!= 0)[0]
    
    model_ML = Pipeline([
    ('clf',LogisticRegression(solver='sag'))
                                                                    ])
    
    print('Fitting model')
    clf=model_ML.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print('\n')
    print(classification_report( y_test, y_pred))
    print((y_pred == y_test).mean())
    
    print('%s features were of importance, the following are the weights:' %len(index))
    print('The top 50 features are:','\n',ISON.get_top_features(feature_scores,index, vectorizer)[:50])



In [10]:
for genre in experiment: 
    print('Logistic Regression Binary Genre experiment for ', genre)
    %time ISON_Film_experiment2(data_N,labels_binary[genre],.20, cv)
    print(88 * '#')

Logistic Regression Binary Genre experiment for  Action
START


/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class RandomizedLogisticRegression is deprecated; The class RandomizedLogisticRegression is deprecated in 0.19 and will be removed in 0.21.
  warnings.warn(msg, category=DeprecationWarning)


Fitting model


/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)




             precision    recall  f1-score   support

     Action       0.76      0.67      0.72        58
 Not_Action       0.88      0.92      0.90       156

avg / total       0.85      0.86      0.85       214

0.8551401869158879
1005 features were of importance, the following are the weights:
The top 50 features are: 
 [(0.55, 'kicks'), (0.54, 'sit'), (0.535, 'bed'), (0.53, 'weapons'), (0.515, 'pursuit'), (0.515, 'lands'), (0.51, 'trained'), (0.51, 'rips'), (0.505, 'fires'), (0.495, 'today'), (0.495, 'dives'), (0.49, 'house'), (0.475, 'tech'), (0.475, 'remaining'), (0.47, 'battle'), (0.47, 'airborne'), (0.465, 'flying'), (0.465, 'blast'), (0.45, 'explodes'), (0.45, 'explode'), (0.445, 'blown'), (0.44, 'blocks'), (0.43, 'steel'), (0.425, 'wreckage'), (0.425, 'weapon'), (0.415, 'bullets'), (0.415, 'armed'), (0.405, 'smashing'), (0.4, 'talk'), (0.39, 'flaming'), (0.375, 'engine'), (0.37, 'sirens'), (0.37, 'rooftop'), (0.365, 'gun'), (0.36, 'destroyed'), (0.35, 'sits'), (0.345, 'cra

...

/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)




               precision    recall  f1-score   support

    Adventure       0.00      0.00      0.00        12
Not_Adventure       0.94      1.00      0.97       202

  avg / total       0.89      0.94      0.92       214

0.9439252336448598
314 features were of importance, the following are the weights:
The top 50 features are: 
 [(0.385, 'woooo'), (0.325, 'epaulet'), (0.32, 'powerline'), (0.315, 'deum'), (0.31, 'explorers'), (0.305, 'winged'), (0.305, 'enchantment'), (0.285, 'swashbucklers'), (0.285, 'fone'), (0.285, 'aaaaarrrrrgggghhhh'), (0.275, 'pod'), (0.275, 'gunstock'), (0.275, 'chilli'), (0.27, 'krumping'), (0.27, 'breakdancing'), (0.265, 'seq'), (0.26, 'pleeeease'), (0.26, 'paean'), (0.26, 'crowbait'), (0.255, 'twoo'), (0.255, 'raaah'), (0.245, 'unemployable'), (0.245, 'ronco'), (0.24, 'aaaaahhhhhh'), (0.235, 'outloud'), (0.235, 'misters'), (0.235, 'enchantress'), (0.235, 'donkeys'), (0.235, 'apu'), (0.235, 'amiability'), (0.23, 'misunder'), (0.23, 'mice'), (0.225, 'meander

/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class RandomizedLogisticRegression is deprecated; The class RandomizedLogisticRegression is deprecated in 0.19 and will be removed in 0.21.
  warnings.warn(msg, category=DeprecationWarning)


Fitting model


             precision    recall  f1-score   support

     Horror       1.00      0.27      0.43        11
 Not_Horror       0.96      1.00      0.98       203

avg / total       0.96      0.96      0.95       214

0.9626168224299065
193 features were of importance, the following are the weights:
The top 50 features are: 
 [(0.4, 'blood'), (0.375, 'screams'), (0.345, 'poltergeists'), (0.345, 'corpse'), (0.335, 'drip'), (0.325, 'clotted'), (0.32, 'boooh'), (0.3, 'snowbound'), (0.295, 'knife'), (0.295, 'hatstand'), (0.295, 'acclimated'), (0.29, 'scream'), (0.29, 'rehangs'), (0.29, 'newsvan'), (0.29, 'carpal'), (0.285, 'neverending'), (0.285, 'deadbolt'), (0.28, 'superbitch'), (0.28, 'appalls'), (0.275, 'shape'), (0.275, 'parapsychologists'), (0.275, 'inebriation'), (0.265, 'weathers'), (0.265, 'samll'), (0.265, 'burts'), (0.26, 'photomat'), (0.26, 'levasseur'), (0.26, 'forecasters'), (0.26, 'defribulator'), (0.255, 'mammalian'), (0.255, 'lipsync'), (0.255, 'cenobites'), (

/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
